In [ ]:
#transactions per second in a lifespan of 1 year
#https://www.blockchain.com/api/charts_api

##Functions

In [ ]:
def group_chunks(df, id_loc, value_loc): #df, 0, 1
  def average(list1):
    sum1 = 0
    for _ in list1:
      sum1 += _
    return sum1/len(list1)

  #convert DataFrame into a dict with a unique value per timestamp
  mydict = {}
  for x in range(len(df)):
    currentid = df.iloc[x,id_loc]
    currentvalue = df.iloc[x,value_loc]
    mydict.setdefault(currentid, [])
    mydict[currentid].append(currentvalue)
  mydict

  #convert dict into a list
  dictlist = list()
  for key, value in mydict.items():
    temp = [key,value]
    dictlist.append(temp)

  #convert to DataFrame
  dictlist = pd.DataFrame(dictlist)
  dictlist

  #average of multiple values
  dictlist[1] = dictlist[1].apply(lambda x : average(x))
  return dictlist

In [ ]:
#***cambiare questa per determinare inizio conteggio settimana
benchmark_date = 2015

import datetime
def convert_week(date):
  year = int(date[0:4])
  return datetime.datetime.strptime(date, '%Y-%m-%d').isocalendar()[1] + (year-benchmark_date)*52
convert_week('2018-01-01')
convert_week('2019-01-01')

def convert_week_back(week_n):
  year_week = int(week_n/52)
  year = benchmark_date + year_week
  d = str(year)+'-W'+str(week_n%52)
  r = datetime.datetime.strptime(d + '-1', "%Y-W%W-%w")
  return r

convert_week_back(129)

datetime.datetime(2017, 6, 19, 0, 0)

In [ ]:
#opening json files from drive, one example
f = open('/content/drive/My Drive/Colab Notebooks/Projects_Work/20200617_Bitcoin/fees-usd-per-transaction.json')
data = json.load(f)
data

##Download Data

In [ ]:
import json
import urllib.request
import pandas as pd

with urllib.request.urlopen('https://api.blockchain.info/charts/market-cap?timespan=5years&rollingAverage=24hours&format=json') as url:
  response = url.read()
  data = json.loads(response)

import pandas as pd
data = pd.DataFrame(data)
data

,status,name,unit,period,description,values
0,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,"{'x': 1436418524, 'y': 3858463442.6750526}"
1,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,"{'x': 1436532268, 'y': 3851885351.6221175}"
2,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,"{'x': 1436640639, 'y': 3915180780.4732704}"
3,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,"{'x': 1436764315, 'y': 4052470543.9329143}"
4,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,"{'x': 1436871065, 'y': 4241715527.078092}"
...,...,...,...,...,...,...
1496,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,"{'x': 1593435074, 'y': 167546872072.3843}"
1497,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,"{'x': 1593531853, 'y': 167482788267.95184}"
1498,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,"{'x': 1593638587, 'y': 168390473144.07108}"
1499,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,"{'x': 1593732338, 'y': 168867619629.73785}"


In [ ]:
import json
import urllib.request
import pandas as pd

def download_data(chart_name, data_=None, compress=True):
  #https://api.blockchain.info/charts/transactions-per-second?timespan=2years&rollingAverage=8hours&format=json

  if data_ is None:
    with urllib.request.urlopen('https://api.blockchain.info/charts/' + chart_name + '?timespan=5years&rollingAverage=24hours&format=json') as url:
      response = url.read()
      data = json.loads(response)
  else:
    data = open(data_)
    data = json.load(data)
  
  import pandas as pd
  data = pd.DataFrame(data)
  data

  #make backup
  data_copy = data.copy()

  def extract_time(dict1):
    dict2 = list(dict1.values())
    #dict2 = list(data['values'][0].values())[0],
    return dict2

  def time_converter(time1):
    from datetime import datetime
    #return datetime.fromtimestamp(time1).strftime('%Y-%m-%d %H:%M:%S')
    return datetime.fromtimestamp(time1).strftime('%Y-%m-%d')

  #preprocessing data
  data = data_copy.copy()
  data['timestamp'] = data['values'].apply(lambda x : extract_time(x)[0])
  data['timestamp'] = data['timestamp'].apply(lambda x : time_converter(x))
  data[chart_name] = data['values'].apply(lambda x : extract_time(x)[1])
  data.pop('values')
  data

  df = data.drop(['status', 'name', 'unit', 'period', 'description'], axis=1)
  df

  if compress == True:
    #group by date
    df = group_chunks(df, 0, 1)
    #df = df.drop(df.index[[730]])
    df

    #apply weeks
    df['week'] = df[0].apply(lambda x : convert_week(x))
    df.pop(0)
    df

    #group by week
    df = group_chunks(df, 1, 0)
    df

  return df

In [ ]:
market_cap = download_data('market-cap')
market_cap.columns = ['week', 'market_cap']
market_cap

,week,market_cap
0,28,3.875177e+09
1,29,4.142558e+09
2,30,4.004468e+09
3,31,4.177792e+09
4,32,4.064795e+09
...,...,...
255,283,1.776837e+11
256,284,1.770631e+11
257,285,1.726850e+11
258,286,1.723651e+11


In [ ]:
#download data compressed by week
market_cap = download_data('market-cap')
tps = download_data('transactions-per-second')
market_price =  download_data('market-price')
total_bitcoins = download_data('total-bitcoins')
miners_revenue = download_data('miners-revenue')

#importing data by json, then compressing
#transaction_fees = download_data('transaction-fees-usd') #url not working
transaction_fees = download_data(0, data_='/content/drive/My Drive/Colab Notebooks/Projects_Work/20200617_Bitcoin/fees-usd-per-transaction.json') #url non funziona

In [ ]:
#if you want to download a single piece of data
market_price =  download_data('market-price', compress=False)
market_price
market_price.to_csv('bitcoin.csv')

##merge all datasets

In [ ]:
#tps, market_cap, total_bitcoins, market_price
result = pd.merge(tps, market_cap, on=0)
result = pd.merge(result, total_bitcoins, on=0)
result = pd.merge(result, market_price, on=0)
result = pd.merge(result, miners_revenue, on=0)
result = pd.merge(result, transaction_fees, on=0)
result

,0,1_x,1_y,1_x,1_y,1_x,1_y
0,75,2.282639,9.040228e+09,1.562784e+07,582.725714,2.319604e+06,0.143732
1,76,2.713914,1.059887e+10,1.565504e+07,716.144286,2.886912e+06,0.222933
2,77,2.497272,1.099238e+10,1.568228e+07,669.740000,2.537684e+06,0.205097
3,78,2.252927,1.019498e+10,1.570727e+07,658.310000,2.617662e+06,0.177134
4,79,2.174107,1.049557e+10,1.573250e+07,661.374286,2.274917e+06,0.170747
...,...,...,...,...,...,...,...
208,283,3.405134,1.766778e+11,1.839208e+07,9703.917143,1.008126e+07,1.788340
209,284,3.479638,1.775381e+11,1.839942e+07,9631.944286,1.032949e+07,0.964811
210,285,3.470908,1.728119e+11,1.840564e+07,9396.455714,8.614685e+06,1.152122
211,286,3.470784,1.726835e+11,1.841185e+07,9324.897143,8.952018e+06,1.025005


In [ ]:
result.columns = ['week', 'tps', 'market_cap', 'total_bitcoins', 'market_price', 'miners_revenue', 'transaction_fees']
result

,week,tps,market_cap,total_bitcoins,market_price,miners_revenue,transaction_fees
0,75,2.282639,9.040228e+09,1.562784e+07,582.725714,2.319604e+06,0.143732
1,76,2.713914,1.059887e+10,1.565504e+07,716.144286,2.886912e+06,0.222933
2,77,2.497272,1.099238e+10,1.568228e+07,669.740000,2.537684e+06,0.205097
3,78,2.252927,1.019498e+10,1.570727e+07,658.310000,2.617662e+06,0.177134
4,79,2.174107,1.049557e+10,1.573250e+07,661.374286,2.274917e+06,0.170747
...,...,...,...,...,...,...,...
208,283,3.405134,1.766778e+11,1.839208e+07,9703.917143,1.008126e+07,1.788340
209,284,3.479638,1.775381e+11,1.839942e+07,9631.944286,1.032949e+07,0.964811
210,285,3.470908,1.728119e+11,1.840564e+07,9396.455714,8.614685e+06,1.152122
211,286,3.470784,1.726835e+11,1.841185e+07,9324.897143,8.952018e+06,1.025005


##CPI

In [ ]:
#https://inflationdata.com/Inflation/Consumer_Price_Index/HistoricalCPI.aspx?reloaded=true
#https://www.usinflationcalculator.com/inflation/consumer-price-index-and-annual-percent-changes-from-1913-to-2008/

In [ ]:
import datetime

#CPI
CPI = [
       '233.707',	 '234.722',	 '236.119',	 '236.599',	 '237.805',	 '238.638',	 '238.654',	 '238.316',	 '237.945',	 '237.838',	 '237.336',	 '236.525',	 
       '236.916',	 '237.111',	 '238.132',	 '239.261',	 '240.236',	 '241.038',	 '240.647',	 '240.853',	 '241.428',	 '241.729',	 '241.353',	 '241.432',	 
       '242.839',	 '243.603',	 '243.801',	 '244.524',	 '244.733',	 '244.955',	 '244.786',	 '245.519',	 '246.819',	 '246.663',	 '246.669',	 '246.524',	 
       '247.867',	 '248.991',	 '249.554',	 '250.546',	 '251.588',	 '251.989',	 '252.006',	 '252.146',	 '252.439',	 '252.885',	 '252.038',	 '251.233',	 
       '251.712',	 '252.776',	 '254.202',	 '255.548',	 '256.092',	 '256.143',	 '256.571',	 '256.558',	 '256.759',	 '257.346',	 '257.208',	 '256.974',	 
       '257.971',	 '258.678',	 '258.115',	 '256.389',	 '256.394', '256.399', '256.405'
      ]
      
CPI = pd.DataFrame(CPI)
CPI

#date
date_list = list()
for year in range(2015, 2020):
  for month in range(1, 13):
    date_list.append(datetime.datetime(year, month, 1).strftime('%Y-%m-%d'))
date_list.append(datetime.datetime(2020, 1, 1).strftime('%Y-%m-%d'))
date_list.append(datetime.datetime(2020, 2, 1).strftime('%Y-%m-%d'))
date_list.append(datetime.datetime(2020, 3, 1).strftime('%Y-%m-%d'))
date_list.append(datetime.datetime(2020, 4, 1).strftime('%Y-%m-%d'))
date_list.append(datetime.datetime(2020, 5, 1).strftime('%Y-%m-%d'))
date_list.append(datetime.datetime(2020, 6, 1).strftime('%Y-%m-%d'))
date_list.append(datetime.datetime(2020, 7, 1).strftime('%Y-%m-%d'))
date_list = pd.DataFrame(date_list)

#concatenation
CPI = pd.concat([date_list, CPI], axis=1)
CPI.columns = ['date', 'CPI']
CPI

,date,CPI
0,2015-01-01,233.707
1,2015-02-01,234.722
2,2015-03-01,236.119
3,2015-04-01,236.599
4,2015-05-01,237.805
...,...,...
62,2020-03-01,258.115
63,2020-04-01,256.389
64,2020-05-01,256.394
65,2020-06-01,256.399


In [ ]:
import datetime
import pandas as pd
base = (datetime.datetime(2015, 1, 1))
date_list = [base + datetime.timedelta(days=x) for x in range(2000)]

date_list_ = list()
for _ in date_list:
  date_list_.append(_.strftime('%Y-%m-%d'))
date_list_[len(date_list_)-1]

date_list = pd.DataFrame(date_list_)
date_list['CPI'] = 0.0
date_list.columns = ['date', 'CPI']
date_list

CPI = date_list.merge(CPI, on='date', how='outer')
#result['CPI_y'].fillna(0, inplace=True)
CPI.pop('CPI_x')
CPI['CPI_y'] = CPI['CPI_y'].values.astype(float)
CPI
CPI['CPI_y'].interpolate(method='polynomial', order=3, inplace=True)
CPI

,date,CPI_y
0,2015-01-01,233.707000
1,2015-01-02,233.701194
2,2015-01-03,233.698951
3,2015-01-04,233.700170
4,2015-01-05,233.704748
...,...,...
1996,2020-06-19,256.384623
1997,2020-06-20,256.388604
1998,2020-06-21,256.393306
1999,2020-06-22,256.398760


In [ ]:
#group by date
CPI = group_chunks(CPI, 0, 1)
CPI

#apply weeks
CPI['week'] = CPI[0].apply(lambda x : convert_week(x))
CPI.pop(0)

#errore in riga 731: valore a posto, ma settimana no
CPI[720:740]

,1,week
720,242.306280,103
721,242.357386,103
722,242.408316,103
723,242.458951,103
724,242.509170,103
725,242.558854,104
726,242.607883,104
727,242.656136,104
728,242.703495,104
729,242.749838,104


In [ ]:
#group by week
CPI = group_chunks(CPI, 1, 0)
CPI.columns = ['week', 'CPI']
CPI[100:110]

,week,CPI
100,100,241.436529
101,101,241.683508
102,102,242.004003
103,103,242.357035
104,104,242.678542
105,156,247.060307
106,106,243.233280
107,107,243.409469
108,108,243.532992
109,109,243.609715


In [ ]:
CPI['week'][105] = 105
CPI['CPI'][105] = 242.878542
CPI[100:110]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,week,CPI
100,100,241.436529
101,101,241.683508
102,102,242.004003
103,103,242.357035
104,104,242.678542
105,105,242.878542
106,106,243.233280
107,107,243.409469
108,108,243.532992
109,109,243.609715


In [ ]:
df = pd.merge(result, CPI, on='week')
df

,week,tps,market_cap,total_bitcoins,market_price,miners_revenue,transaction_fees,CPI
0,75,2.282639,9.040228e+09,1.562784e+07,582.725714,2.319604e+06,0.143732,240.995896
1,76,2.713914,1.059887e+10,1.565504e+07,716.144286,2.886912e+06,0.222933,240.890890
2,77,2.497272,1.099238e+10,1.568228e+07,669.740000,2.537684e+06,0.205097,240.764425
3,78,2.252927,1.019498e+10,1.570727e+07,658.310000,2.617662e+06,0.177134,240.660392
4,79,2.174107,1.049557e+10,1.573250e+07,661.374286,2.274917e+06,0.170747,240.614773
...,...,...,...,...,...,...,...,...
208,283,3.405134,1.766778e+11,1.839208e+07,9703.917143,1.008126e+07,1.788340,256.389586
209,284,3.479638,1.775381e+11,1.839942e+07,9631.944286,1.032949e+07,0.964811,256.375644
210,285,3.470908,1.728119e+11,1.840564e+07,9396.455714,8.614685e+06,1.152122,256.382639
211,286,3.470784,1.726835e+11,1.841185e+07,9324.897143,8.952018e+06,1.025005,256.398760


In [ ]:
df = df.drop(df.index[[30]])
df

,week,tps,market_cap,total_bitcoins,market_price,miners_revenue,transaction_fees,CPI
0,75,2.282639,9.040228e+09,1.562784e+07,582.725714,2.319604e+06,0.143732,240.995896
1,76,2.713914,1.059887e+10,1.565504e+07,716.144286,2.886912e+06,0.222933,240.890890
2,77,2.497272,1.099238e+10,1.568228e+07,669.740000,2.537684e+06,0.205097,240.764425
3,78,2.252927,1.019498e+10,1.570727e+07,658.310000,2.617662e+06,0.177134,240.660392
4,79,2.174107,1.049557e+10,1.573250e+07,661.374286,2.274917e+06,0.170747,240.614773
...,...,...,...,...,...,...,...,...
208,283,3.405134,1.766778e+11,1.839208e+07,9703.917143,1.008126e+07,1.788340,256.389586
209,284,3.479638,1.775381e+11,1.839942e+07,9631.944286,1.032949e+07,0.964811,256.375644
210,285,3.470908,1.728119e+11,1.840564e+07,9396.455714,8.614685e+06,1.152122,256.382639
211,286,3.470784,1.726835e+11,1.841185e+07,9324.897143,8.952018e+06,1.025005,256.398760


In [ ]:
df = df.reset_index()
df.pop('index')
df

,week,tps,market_cap,total_bitcoins,market_price,miners_revenue,transaction_fees,CPI
0,75,2.282639,9.040228e+09,1.562784e+07,582.725714,2.319604e+06,0.143732,240.995896
1,76,2.713914,1.059887e+10,1.565504e+07,716.144286,2.886912e+06,0.222933,240.890890
2,77,2.497272,1.099238e+10,1.568228e+07,669.740000,2.537684e+06,0.205097,240.764425
3,78,2.252927,1.019498e+10,1.570727e+07,658.310000,2.617662e+06,0.177134,240.660392
4,79,2.174107,1.049557e+10,1.573250e+07,661.374286,2.274917e+06,0.170747,240.614773
...,...,...,...,...,...,...,...,...
207,283,3.405134,1.766778e+11,1.839208e+07,9703.917143,1.008126e+07,1.788340,256.389586
208,284,3.479638,1.775381e+11,1.839942e+07,9631.944286,1.032949e+07,0.964811,256.375644
209,285,3.470908,1.728119e+11,1.840564e+07,9396.455714,8.614685e+06,1.152122,256.382639
210,286,3.470784,1.726835e+11,1.841185e+07,9324.897143,8.952018e+06,1.025005,256.398760


In [ ]:
df['timestamp'] = df['week'].apply(lambda x : convert_week_back(x))
df

,week,tps,market_cap,total_bitcoins,market_price,miners_revenue,transaction_fees,CPI,timestamp
0,75,2.282639,9.040228e+09,1.562784e+07,582.725714,2.319604e+06,0.143732,240.995896,2016-06-06
1,76,2.713914,1.059887e+10,1.565504e+07,716.144286,2.886912e+06,0.222933,240.890890,2016-06-13
2,77,2.497272,1.099238e+10,1.568228e+07,669.740000,2.537684e+06,0.205097,240.764425,2016-06-20
3,78,2.252927,1.019498e+10,1.570727e+07,658.310000,2.617662e+06,0.177134,240.660392,2016-06-27
4,79,2.174107,1.049557e+10,1.573250e+07,661.374286,2.274917e+06,0.170747,240.614773,2016-07-04
...,...,...,...,...,...,...,...,...,...
207,283,3.405134,1.766778e+11,1.839208e+07,9703.917143,1.008126e+07,1.788340,256.389586,2020-06-08
208,284,3.479638,1.775381e+11,1.839942e+07,9631.944286,1.032949e+07,0.964811,256.375644,2020-06-15
209,285,3.470908,1.728119e+11,1.840564e+07,9396.455714,8.614685e+06,1.152122,256.382639,2020-06-22
210,286,3.470784,1.726835e+11,1.841185e+07,9324.897143,8.952018e+06,1.025005,256.398760,2020-06-29


In [ ]:
df.to_csv('bitcoin.csv')

#Procedure Example

In [ ]:
import json
import urllib.request

#https://api.blockchain.info/charts/transactions-per-second?timespan=2years&rollingAverage=8hours&format=json
with urllib.request.urlopen("https://api.blockchain.info/charts/market-cap?timespan=2years&rollingAverage=24hours&format=json") as url:
  response = url.read()
  data = json.loads(response)

In [ ]:
import pandas as pd
data = pd.DataFrame(data)
data

,status,name,unit,period,description,values
0,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,"{'x': 1532033668, 'y': 113149405085.26335}"
1,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,"{'x': 1532071949, 'y': 113626741608.18709}"
2,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,"{'x': 1532105067, 'y': 114123325317.08546}"
3,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,"{'x': 1532146275, 'y': 114565727637.77759}"
4,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,"{'x': 1532189461, 'y': 114921195536.95058}"
...,...,...,...,...,...,...
1500,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,"{'x': 1593376920, 'y': 172588083178.22244}"
1501,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,"{'x': 1593419815, 'y': 172238575184.7884}"
1502,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,"{'x': 1593458191, 'y': 171892578412.20844}"
1503,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,"{'x': 1593493811, 'y': 171790844584.68216}"


In [ ]:
#make backup
data_copy = data.copy()

In [ ]:
def extract_time(dict1):
  dict2 = list(dict1.values())
  #dict2 = list(data['values'][0].values())[0],
  return dict2

def time_converter(time1):
  from datetime import datetime
  #return datetime.fromtimestamp(time1).strftime('%Y-%m-%d %H:%M:%S')
  return datetime.fromtimestamp(time1).strftime('%Y-%m-%d')

In [ ]:
data = data_copy.copy()
data['timestamp'] = data['values'].apply(lambda x : extract_time(x)[0])
data['timestamp'] = data['timestamp'].apply(lambda x : time_converter(x))
data['market cap'] = data['values'].apply(lambda x : extract_time(x)[1])
data.pop('values')
data

,status,name,unit,period,description,timestamp,market cap
0,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,2018-07-19,1.131494e+11
1,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,2018-07-20,1.136267e+11
2,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,2018-07-20,1.141233e+11
3,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,2018-07-21,1.145657e+11
4,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,2018-07-21,1.149212e+11
...,...,...,...,...,...,...,...
1500,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,2020-06-28,1.725881e+11
1501,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,2020-06-29,1.722386e+11
1502,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,2020-06-29,1.718926e+11
1503,ok,Market Capitalization,USD,day,The total USD value of bitcoin supply in circu...,2020-06-30,1.717908e+11
